In [0]:
print("Hello")

Hello



#### Sending test event to event hub 

In [0]:
from azure.eventhub import EventHubProducerClient, EventData
import json
# event hub config 

event_hub_connection_string= 'Endpoint=sb://apistreamingnamespace.servicebus.windows.net/;SharedAccessKeyName=fordatabricks;SharedAccessKey=zj+rHq2kk8P4aIAgsLgR4z122Xs2Tsa4S+AEhKyAy4Y=;EntityPath=weatherstreamingeventhub'
event_hub_name = 'weatherstreamingeventhub'

# initialize evnt hub producer 

producer = EventHubProducerClient.from_connection_string(conn_str=event_hub_connection_string, eventhub_name=event_hub_name)

# send event to event hub
def send_event(event):
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json.dumps(event)))
    producer.send_batch(event_data_batch)

event = {"name": " Kumar", "age": 39}
send_event(event)

producer.close()


In [0]:
from azure.eventhub import EventHubProducerClient, EventData
import json
# event hub config 

eventhub_connection_string = dbutils.secrets.get(scope="key-vault-scope", key="eventhub-connection-string")
event_hub_name = 'weatherstreamingeventhub'

# initialize evnt hub producer 

producer = EventHubProducerClient.from_connection_string(conn_str=eventhub_connection_string, eventhub_name=event_hub_name)

# send event to event hub
def send_event(event):
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json.dumps(event)))
    producer.send_batch(event_data_batch)

event = {"name": " Jiyan", "age": 6}
send_event(event)

producer.close()



### Weather API Initial 

In [0]:
import requests
import json 

weatherapi_key = dbutils.secrets.get(scope="key-vault-scope", key="weatherapilatest")
location = 'Bangalore'
base_url = 'http://api.weatherapi.com/v1'

current_weather_url = f"{base_url}/current.json?key={weatherapi_key}&q={location}&aqi=yes"
response = requests.get(current_weather_url)
if response.status_code == 200:
    data = response.json()
    print(json.dumps(data,indent=3))
else:
    print(f"error : {response.status_code}, {response.text}")


{
   "location": {
      "name": "Bangalore",
      "region": "Karnataka",
      "country": "India",
      "lat": 12.9833,
      "lon": 77.5833,
      "tz_id": "Asia/Kolkata",
      "localtime_epoch": 1756743007,
      "localtime": "2025-09-01 21:40"
   },
   "current": {
      "last_updated_epoch": 1756742400,
      "last_updated": "2025-09-01 21:30",
      "temp_c": 21.1,
      "temp_f": 70.0,
      "is_day": 0,
      "condition": {
         "text": "Partly cloudy",
         "icon": "//cdn.weatherapi.com/weather/64x64/night/116.png",
         "code": 1003
      },
      "wind_mph": 12.3,
      "wind_kph": 19.8,
      "wind_degree": 265,
      "wind_dir": "W",
      "pressure_mb": 1013.0,
      "pressure_in": 29.91,
      "precip_mm": 0.0,
      "precip_in": 0.0,
      "humidity": 94,
      "cloud": 75,
      "feelslike_c": 21.1,
      "feelslike_f": 70.0,
      "windchill_c": 21.1,
      "windchill_f": 69.9,
      "heatindex_c": 24.2,
      "heatindex_f": 75.6,
      "dewpoint_c": 18

#### Complete code for weather data 

In [0]:
import requests
import json

def api_response(response):
    if response.status_code == 200:
        data = response.json()
        return data 
    else: 
        return f"error : {response.status_code}, {response.text}"

def current_weather_data(base_url, weather_key,location):
    current_weather_url = f"{base_url}/current.json?key={weatherapi_key}&q={location}&aqi=yes"
    response = requests.get(current_weather_url)
    return api_response(response)

def forecast_weather_data(base_url, weather_key,location):
    forecast_weather_url = f"{base_url}/forecast.json?key={weatherapi_key}&q={location}&days=3&aqi=yes&alerts=no"
    response = requests.get(forecast_weather_url)
    return api_response(response)

def alert_weather_data(base_url, weather_key,location):
    alert_weather_url = f"{base_url}/alerts.json?key={weatherapi_key}&q={location}&days=3&aqi=yes&alerts=yes"
    response = requests.get(alert_weather_url)
    return api_response(response)

def flatten_merge(current_weather,forecast_weather,alert_weather): 
    location_data = current_weather.get('location', {})
    current = current_weather.get('current', {})
    condition = current.get('condition', {})
    air_quality = current.get('air_quality',{})
    forecast = forecast_weather['forecast']['forecastday']
    #alerts = alert_weather.get('alerts', []) if isinstance(alert_weather, dict) else []
    alerts = alert_weather.get('alerts', {}).get('alerts', [])
    flattened_data = {
        'name': location_data.get('name'),
        'country': location_data.get('country'),
        'region': location_data.get('region'),
        'lat': location_data.get('lat'),
        'lon': location_data.get('lon'),
        'localtime_epoch': location_data.get('localtime_epoch'),
        'localtime': location_data.get('localtime'),
        'condition_text': condition.get('text'),
        'condition_icon': condition.get('icon'),
        'condition_code': condition.get('code'),
        'wind_kph': current.get('wind_kph'),
        'wind_degree': current.get('wind_degree'),
        'wind_dir': current.get('wind_dir'),
        'humidity': current.get('humidity'),
        'feelslike_c': current.get('feelslike_c'),
        'uv': current.get('uv'),
        'gust_kph': current.get('gust_kph'),
        'air_quality': {
            'co': air_quality.get('co'),
            'no2': air_quality.get('no2'),
            'o3': air_quality.get('o3'),
            'so2': air_quality.get('so2'),
            'pm2_5': air_quality.get('pm2_5'), 
            'pm10': air_quality.get('pm10'),
            'us-epa-index': air_quality.get('us-epa-index'),
            'gb-defra-index': air_quality.get('gb-defra-index')
        },
        'alerts' : [
            {
            'headline': alert.get('headline'),
            'severity': alert.get('severity'),
            'desc': alert.get('desc'),
            'date_epoch': alert.get('date_epoch'),
            'date': alert.get('date'),
            'status': alert.get('status'),
            'url': alert.get('url'),
            'event': alert.get('event'),
            'type': alert.get('type')
        } 
        for alert in alerts
        ], 
        'forecast' : [ 
        {
            'date': day.get('date'),
            'date_epoch': day.get('day',{}).get('date_epoch'),
            'maxtemp_c': day.get('day',{}).get('maxtemp_c'),
            'maxtemp_f': day.get('day',{}).get('maxtemp_f'),
            'mintemp_c': day.get('day',{}).get('mintemp_c'),
            'mintemp_f': day.get('day',{}).get('mintemp_f'),
            'condition': day.get('day',{}).get('condition',{}).get('text'),
            'condition_icon': day.get('day',{}).get('condition',{}).get('icon')
        } for day in forecast
        ]
    }
    return flattened_data

def fetch_data():
    weatherapi_key = dbutils.secrets.get(scope="key-vault-scope", key="weatherapilatest")
    location = 'Bangalore'
    base_url = 'http://api.weatherapi.com/v1'
    current_weather = current_weather_data(base_url, weatherapi_key,location)
    forecast_weather = forecast_weather_data(base_url, weatherapi_key,location)
    alert_weather = alert_weather_data(base_url, weatherapi_key,location)
    flattened_data = flatten_merge(current_weather,forecast_weather,alert_weather)
    print(json.dumps(flattened_data,indent=3))
    return flattened_data

fetch_data()

    





{
   "name": "Bangalore",
   "country": "India",
   "region": "Karnataka",
   "lat": 12.9833,
   "lon": 77.5833,
   "localtime_epoch": 1756745695,
   "localtime": "2025-09-01 22:24",
   "condition_text": "Partly cloudy",
   "condition_icon": "//cdn.weatherapi.com/weather/64x64/night/116.png",
   "condition_code": 1003,
   "wind_kph": 19.8,
   "wind_degree": 267,
   "wind_dir": "W",
   "humidity": 94,
   "feelslike_c": 21.2,
   "uv": 0.0,
   "gust_kph": 28.4,
   "air_quality": {
      "co": 817.7,
      "no2": 31.45,
      "o3": 45.0,
      "so2": 9.25,
      "pm2_5": 17.205,
      "pm10": 24.975,
      "us-epa-index": 2,
      "gb-defra-index": 2
   },
   "alerts": [],
   "forecast": [
      {
         "date": "2025-09-01",
         "date_epoch": null,
         "maxtemp_c": 27.0,
         "maxtemp_f": 80.5,
         "mintemp_c": 20.1,
         "mintemp_f": 68.1,
         "condition": "Patchy rain nearby",
         "condition_icon": "//cdn.weatherapi.com/weather/64x64/day/176.png"
     

{'name': 'Bangalore',
 'country': 'India',
 'region': 'Karnataka',
 'lat': 12.9833,
 'lon': 77.5833,
 'localtime_epoch': 1756745695,
 'localtime': '2025-09-01 22:24',
 'condition_text': 'Partly cloudy',
 'condition_icon': '//cdn.weatherapi.com/weather/64x64/night/116.png',
 'condition_code': 1003,
 'wind_kph': 19.8,
 'wind_degree': 267,
 'wind_dir': 'W',
 'humidity': 94,
 'feelslike_c': 21.2,
 'uv': 0.0,
 'gust_kph': 28.4,
 'air_quality': {'co': 817.7,
  'no2': 31.45,
  'o3': 45.0,
  'so2': 9.25,
  'pm2_5': 17.205,
  'pm10': 24.975,
  'us-epa-index': 2,
  'gb-defra-index': 2},
 'alerts': [],
 'forecast': [{'date': '2025-09-01',
   'date_epoch': None,
   'maxtemp_c': 27.0,
   'maxtemp_f': 80.5,
   'mintemp_c': 20.1,
   'mintemp_f': 68.1,
   'condition': 'Patchy rain nearby',
   'condition_icon': '//cdn.weatherapi.com/weather/64x64/day/176.png'},
  {'date': '2025-09-02',
   'date_epoch': None,
   'maxtemp_c': 25.4,
   'maxtemp_f': 77.8,
   'mintemp_c': 19.5,
   'mintemp_f': 67.1,
   'con

#### Sending weather data to event hub 

In [0]:
import requests
import json
from azure.eventhub import EventHubProducerClient, EventData

eventhub_connection_string = dbutils.secrets.get(scope="key-vault-scope", key="eventhub-connection-string")
event_hub_name = 'weatherstreamingeventhub'

# initialize evnt hub producer 

producer = EventHubProducerClient.from_connection_string(conn_str=eventhub_connection_string, eventhub_name=event_hub_name)
def send_event(event):
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json.dumps(event)))
    producer.send_batch(event_data_batch)

def api_response(response):
    if response.status_code == 200:
        data = response.json()
        return data 
    else: 
        return f"error : {response.status_code}, {response.text}"

def current_weather_data(base_url, weather_key,location):
    current_weather_url = f"{base_url}/current.json?key={weatherapi_key}&q={location}&aqi=yes"
    response = requests.get(current_weather_url)
    return api_response(response)

def forecast_weather_data(base_url, weather_key,location):
    forecast_weather_url = f"{base_url}/forecast.json?key={weatherapi_key}&q={location}&days=3&aqi=yes&alerts=no"
    response = requests.get(forecast_weather_url)
    return api_response(response)

def alert_weather_data(base_url, weather_key,location):
    alert_weather_url = f"{base_url}/alerts.json?key={weatherapi_key}&q={location}&days=3&aqi=yes&alerts=yes"
    response = requests.get(alert_weather_url)
    return api_response(response)

def flatten_merge(current_weather,forecast_weather,alert_weather): 
    location_data = current_weather.get('location', {})
    current = current_weather.get('current', {})
    condition = current.get('condition', {})
    air_quality = current.get('air_quality',{})
    forecast = forecast_weather['forecast']['forecastday']
    #alerts = alert_weather.get('alerts', []) if isinstance(alert_weather, dict) else []
    alerts = alert_weather.get('alerts', {}).get('alerts', [])
    flattened_data = {
        'name': location_data.get('name'),
        'country': location_data.get('country'),
        'region': location_data.get('region'),
        'lat': location_data.get('lat'),
        'lon': location_data.get('lon'),
        'localtime_epoch': location_data.get('localtime_epoch'),
        'localtime': location_data.get('localtime'),
        'condition_text': condition.get('text'),
        'condition_icon': condition.get('icon'),
        'condition_code': condition.get('code'),
        'wind_kph': current.get('wind_kph'),
        'wind_degree': current.get('wind_degree'),
        'wind_dir': current.get('wind_dir'),
        'humidity': current.get('humidity'),
        'feelslike_c': current.get('feelslike_c'),
        'uv': current.get('uv'),
        'gust_kph': current.get('gust_kph'),
        'air_quality': {
            'co': air_quality.get('co'),
            'no2': air_quality.get('no2'),
            'o3': air_quality.get('o3'),
            'so2': air_quality.get('so2'),
            'pm2_5': air_quality.get('pm2_5'), 
            'pm10': air_quality.get('pm10'),
            'us-epa-index': air_quality.get('us-epa-index'),
            'gb-defra-index': air_quality.get('gb-defra-index')
        },
        'alerts' : [
            {
            'headline': alert.get('headline'),
            'severity': alert.get('severity'),
            'desc': alert.get('desc'),
            'date_epoch': alert.get('date_epoch'),
            'date': alert.get('date'),
            'status': alert.get('status'),
            'url': alert.get('url'),
            'event': alert.get('event'),
            'type': alert.get('type')
        } 
        for alert in alerts
        ], 
        'forecast' : [ 
        {
            'date': day.get('date'),
            'date_epoch': day.get('day',{}).get('date_epoch'),
            'maxtemp_c': day.get('day',{}).get('maxtemp_c'),
            'maxtemp_f': day.get('day',{}).get('maxtemp_f'),
            'mintemp_c': day.get('day',{}).get('mintemp_c'),
            'mintemp_f': day.get('day',{}).get('mintemp_f'),
            'condition': day.get('day',{}).get('condition',{}).get('text'),
            'condition_icon': day.get('day',{}).get('condition',{}).get('icon')
        } for day in forecast
        ]
    }
    return flattened_data

def fetch_data():
    weatherapi_key = dbutils.secrets.get(scope="key-vault-scope", key="weatherapilatest")
    location = 'Bangalore'
    base_url = 'http://api.weatherapi.com/v1'
    current_weather = current_weather_data(base_url, weatherapi_key,location)
    forecast_weather = forecast_weather_data(base_url, weatherapi_key,location)
    alert_weather = alert_weather_data(base_url, weatherapi_key,location)
    flattened_data = flatten_merge(current_weather,forecast_weather,alert_weather)
    send_event(flattened_data)
    return flattened_data

fetch_data()

{'name': 'Bangalore',
 'country': 'India',
 'region': 'Karnataka',
 'lat': 12.9833,
 'lon': 77.5833,
 'localtime_epoch': 1756746435,
 'localtime': '2025-09-01 22:37',
 'condition_text': 'Partly cloudy',
 'condition_icon': '//cdn.weatherapi.com/weather/64x64/night/116.png',
 'condition_code': 1003,
 'wind_kph': 19.8,
 'wind_degree': 267,
 'wind_dir': 'W',
 'humidity': 94,
 'feelslike_c': 21.2,
 'uv': 0.0,
 'gust_kph': 28.4,
 'air_quality': {'co': 817.7,
  'no2': 31.45,
  'o3': 45.0,
  'so2': 9.25,
  'pm2_5': 17.205,
  'pm10': 24.975,
  'us-epa-index': 2,
  'gb-defra-index': 2},
 'alerts': [],
 'forecast': [{'date': '2025-09-01',
   'date_epoch': None,
   'maxtemp_c': 27.0,
   'maxtemp_f': 80.5,
   'mintemp_c': 20.1,
   'mintemp_f': 68.1,
   'condition': 'Patchy rain nearby',
   'condition_icon': '//cdn.weatherapi.com/weather/64x64/day/176.png'},
  {'date': '2025-09-02',
   'date_epoch': None,
   'maxtemp_c': 25.4,
   'maxtemp_f': 77.8,
   'mintemp_c': 19.5,
   'mintemp_f': 67.1,
   'con

#### sending weather data in streaming 

In [0]:
import requests
import json
from azure.eventhub import EventHubProducerClient, EventData

eventhub_connection_string = dbutils.secrets.get(scope="key-vault-scope", key="eventhub-connection-string")
event_hub_name = 'weatherstreamingeventhub'

# initialize evnt hub producer 

producer = EventHubProducerClient.from_connection_string(conn_str=eventhub_connection_string, eventhub_name=event_hub_name)
def send_event(event):
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json.dumps(event)))
    producer.send_batch(event_data_batch)

def api_response(response):
    if response.status_code == 200:
        data = response.json()
        return data 
    else: 
        return f"error : {response.status_code}, {response.text}"

def current_weather_data(base_url, weather_key,location):
    current_weather_url = f"{base_url}/current.json?key={weatherapi_key}&q={location}&aqi=yes"
    response = requests.get(current_weather_url)
    return api_response(response)

def forecast_weather_data(base_url, weather_key,location):
    forecast_weather_url = f"{base_url}/forecast.json?key={weatherapi_key}&q={location}&days=3&aqi=yes&alerts=no"
    response = requests.get(forecast_weather_url)
    return api_response(response)

def alert_weather_data(base_url, weather_key,location):
    alert_weather_url = f"{base_url}/alerts.json?key={weatherapi_key}&q={location}&days=3&aqi=yes&alerts=yes"
    response = requests.get(alert_weather_url)
    return api_response(response)

def flatten_merge(current_weather,forecast_weather,alert_weather): 
    location_data = current_weather.get('location', {})
    current = current_weather.get('current', {})
    condition = current.get('condition', {})
    air_quality = current.get('air_quality',{})
    forecast = forecast_weather['forecast']['forecastday']
    #alerts = alert_weather.get('alerts', []) if isinstance(alert_weather, dict) else []
    alerts = alert_weather.get('alerts', {}).get('alerts', [])
    flattened_data = {
        'name': location_data.get('name'),
        'country': location_data.get('country'),
        'region': location_data.get('region'),
        'lat': location_data.get('lat'),
        'lon': location_data.get('lon'),
        'localtime_epoch': location_data.get('localtime_epoch'),
        'localtime': location_data.get('localtime'),
        'condition_text': condition.get('text'),
        'condition_icon': condition.get('icon'),
        'condition_code': condition.get('code'),
        'wind_kph': current.get('wind_kph'),
        'wind_degree': current.get('wind_degree'),
        'wind_dir': current.get('wind_dir'),
        'humidity': current.get('humidity'),
        'feelslike_c': current.get('feelslike_c'),
        'uv': current.get('uv'),
        'gust_kph': current.get('gust_kph'),
        'air_quality': {
            'co': air_quality.get('co'),
            'no2': air_quality.get('no2'),
            'o3': air_quality.get('o3'),
            'so2': air_quality.get('so2'),
            'pm2_5': air_quality.get('pm2_5'), 
            'pm10': air_quality.get('pm10'),
            'us-epa-index': air_quality.get('us-epa-index'),
            'gb-defra-index': air_quality.get('gb-defra-index')
        },
        'alerts' : [
            {
            'headline': alert.get('headline'),
            'severity': alert.get('severity'),
            'desc': alert.get('desc'),
            'date_epoch': alert.get('date_epoch'),
            'date': alert.get('date'),
            'status': alert.get('status'),
            'url': alert.get('url'),
            'event': alert.get('event'),
            'type': alert.get('type')
        } 
        for alert in alerts
        ], 
        'forecast' : [ 
        {
            'date': day.get('date'),
            'date_epoch': day.get('day',{}).get('date_epoch'),
            'maxtemp_c': day.get('day',{}).get('maxtemp_c'),
            'maxtemp_f': day.get('day',{}).get('maxtemp_f'),
            'mintemp_c': day.get('day',{}).get('mintemp_c'),
            'mintemp_f': day.get('day',{}).get('mintemp_f'),
            'condition': day.get('day',{}).get('condition',{}).get('text'),
            'condition_icon': day.get('day',{}).get('condition',{}).get('icon')
        } for day in forecast
        ]
    }
    return flattened_data

def fetch_data():
    weatherapi_key = dbutils.secrets.get(scope="key-vault-scope", key="weatherapilatest")
    location = 'Bangalore'
    base_url = 'http://api.weatherapi.com/v1'
    current_weather = current_weather_data(base_url, weatherapi_key,location)
    forecast_weather = forecast_weather_data(base_url, weatherapi_key,location)
    alert_weather = alert_weather_data(base_url, weatherapi_key,location)
    flattened_data = flatten_merge(current_weather,forecast_weather,alert_weather)
    
    return flattened_data

def process_batch(batch_df, batch_id): 
    try : 
        stream_data = fetch_data()
        send_event(stream_data)
    except Exception as e:
        print(f"error sending stream data to event hub {batch_id}: str(e)") 
        raise e

streaming_df = spark.readStream.format('rate').option("rowsPerSeconds",1).load()
query = streaming_df.writeStream.foreachBatch(process_batch).start()
query.awaitTermination()
producer.close()


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

##### Sending weather data every 30 seconds 

In [0]:
import requests
import json
from azure.eventhub import EventHubProducerClient, EventData
from datetime import datetime, timedelta

eventhub_connection_string = dbutils.secrets.get(scope="key-vault-scope", key="eventhub-connection-string")
event_hub_name = 'weatherstreamingeventhub'

# initialize evnt hub producer 

producer = EventHubProducerClient.from_connection_string(conn_str=eventhub_connection_string, eventhub_name=event_hub_name)
def send_event(event):
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json.dumps(event)))
    producer.send_batch(event_data_batch)

def api_response(response):
    if response.status_code == 200:
        data = response.json()
        return data 
    else: 
        return f"error : {response.status_code}, {response.text}"

def current_weather_data(base_url, weather_key,location):
    current_weather_url = f"{base_url}/current.json?key={weatherapi_key}&q={location}&aqi=yes"
    response = requests.get(current_weather_url)
    return api_response(response)

def forecast_weather_data(base_url, weather_key,location):
    forecast_weather_url = f"{base_url}/forecast.json?key={weatherapi_key}&q={location}&days=3&aqi=yes&alerts=no"
    response = requests.get(forecast_weather_url)
    return api_response(response)

def alert_weather_data(base_url, weather_key,location):
    alert_weather_url = f"{base_url}/alerts.json?key={weatherapi_key}&q={location}&days=3&aqi=yes&alerts=yes"
    response = requests.get(alert_weather_url)
    return api_response(response)

def flatten_merge(current_weather,forecast_weather,alert_weather): 
    location_data = current_weather.get('location', {})
    current = current_weather.get('current', {})
    condition = current.get('condition', {})
    air_quality = current.get('air_quality',{})
    forecast = forecast_weather['forecast']['forecastday']
    #alerts = alert_weather.get('alerts', []) if isinstance(alert_weather, dict) else []
    alerts = alert_weather.get('alerts', {}).get('alerts', [])
    flattened_data = {
        'name': location_data.get('name'),
        'country': location_data.get('country'),
        'region': location_data.get('region'),
        'lat': location_data.get('lat'),
        'lon': location_data.get('lon'),
        'localtime_epoch': location_data.get('localtime_epoch'),
        'localtime': location_data.get('localtime'),
        'condition_text': condition.get('text'),
        'condition_icon': condition.get('icon'),
        'condition_code': condition.get('code'),
        'wind_kph': current.get('wind_kph'),
        'wind_degree': current.get('wind_degree'),
        'wind_dir': current.get('wind_dir'),
        'humidity': current.get('humidity'),
        'feelslike_c': current.get('feelslike_c'),
        'uv': current.get('uv'),
        'gust_kph': current.get('gust_kph'),
        'air_quality': {
            'co': air_quality.get('co'),
            'no2': air_quality.get('no2'),
            'o3': air_quality.get('o3'),
            'so2': air_quality.get('so2'),
            'pm2_5': air_quality.get('pm2_5'), 
            'pm10': air_quality.get('pm10'),
            'us-epa-index': air_quality.get('us-epa-index'),
            'gb-defra-index': air_quality.get('gb-defra-index')
        },
        'alerts' : [
            {
            'headline': alert.get('headline'),
            'severity': alert.get('severity'),
            'desc': alert.get('desc'),
            'date_epoch': alert.get('date_epoch'),
            'date': alert.get('date'),
            'status': alert.get('status'),
            'url': alert.get('url'),
            'event': alert.get('event'),
            'type': alert.get('type')
        } 
        for alert in alerts
        ], 
        'forecast' : [ 
        {
            'date': day.get('date'),
            'date_epoch': day.get('day',{}).get('date_epoch'),
            'maxtemp_c': day.get('day',{}).get('maxtemp_c'),
            'maxtemp_f': day.get('day',{}).get('maxtemp_f'),
            'mintemp_c': day.get('day',{}).get('mintemp_c'),
            'mintemp_f': day.get('day',{}).get('mintemp_f'),
            'condition': day.get('day',{}).get('condition',{}).get('text'),
            'condition_icon': day.get('day',{}).get('condition',{}).get('icon')
        } for day in forecast
        ]
    }
    return flattened_data



def fetch_data():
    weatherapi_key = dbutils.secrets.get(scope="key-vault-scope", key="weatherapilatest")
    location = 'Bangalore'
    base_url = 'http://api.weatherapi.com/v1'
    current_weather = current_weather_data(base_url, weatherapi_key,location)
    forecast_weather = forecast_weather_data(base_url, weatherapi_key,location)
    alert_weather = alert_weather_data(base_url, weatherapi_key,location)
    flattened_data = flatten_merge(current_weather,forecast_weather,alert_weather)
    
    return flattened_data

last_sent_time = datetime.now() - timedelta(seconds=30)
def process_batch(batch_df, batch_id): 
    global last_sent_time
    try : 
        #last_sent_time = datetime.now() - timedelta(seconds=60)

        current_time = datetime.now()
        if (current_time - last_sent_time ).total_seconds() >=30:
            stream_data = fetch_data()
            send_event(stream_data)
            last_sent_time = current_time
            print(f"event sent time at: {last_sent_time} ")

    except Exception as e:
        print(f"error sending stream data to event hub {batch_id}: str(e)") 
        raise e

streaming_df = spark.readStream.format('rate').option("rowsPerSeconds",1).load()
query = streaming_df.writeStream.foreachBatch(process_batch).start()
query.awaitTermination()
producer.close()


event sent time at: 2025-09-01 17:42:31.467885 
event sent time at: 2025-09-01 17:43:02.180238 
event sent time at: 2025-09-01 17:43:32.261015 


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can